In [88]:
import pandas as pd

# TODO: Process traditional and tourist datasets


class Property:

    RENOVATION_COST_PRICES = {1:50, 2:66, 3:83, 4:100}
    CADASTRAL_MODIFIER = 0.4
    MUNICIPAL_TAX_MODIFIER = 0.00479
    PRICE_DATASET = None
    MANAGEMENT_FEE = 0
    AVG_COMMUNITY_COSTS = 150

    def __init__(self, zone, bedrooms, acquisition_price):
        self.bedrooms = bedrooms
        self.zone = zone
        self.acquisition_price = acquisition_price
        self.municipal_tax = (self.acquisition_price * Property.CADASTRAL_MODIFIER * Property.MUNICIPAL_TAX_MODIFIER) / 12
        self.renovation_cost = Property.RENOVATION_COST_PRICES[self.bedrooms]
        self.community_cost = Property.AVG_COMMUNITY_COSTS
        self.internet_price = 0
        self.price_dataset = Property.PRICE_DATASET
        self.management_fee = Property.MANAGEMENT_FEE

    @property
    def investment(self):
        investment_costs = [self.acquisition_price, self.decoration_price, self.equipment_price]
        return sum(investment_costs)

    @property
    def opex(self):
        opex_costs = [self.utilities_price, self.hotelery_price,
                      self.internet_price, self.municipal_tax, self.renovation_cost, self.community_cost]
        return sum(opex_costs) * self.management_fee

    @property
    def noi(self):
        return (self.average_rental_price * self.avg_occupancy) - self.opex

    @property
    def profitability(self):
        return ((self.noi * 12) / self.investment) * 100



class CorporateProperty(Property):

    RENOVATION_COST_PRICES = {1:50, 2:66, 3:83, 4:100}
    UTILITIES_PRICES = {1:120, 2:150, 3:170, 4:200}
    DECORATION_PRICES = {1:12000, 2:14000, 3:17000, 4:20000}
    EQUIPMENT_PRICES = {1:1200, 2:1400, 3:1700, 4:2000}
    HOTELERY_PRICES = {1:108, 2:150, 3:220, 4:280}
    INTERNET_PRICE = 50
    PRICE_DATASET = 'rental_prices_corporate.xlsx'
    PRICE_DATASET_ZONE_COLUMN = 'LOCATIONNAME'
    PRICE_DATASET_ROOMS_COLUMN = 'ROOMS'
    PRICE_DATASET_PRICE_COLUMN = 'PRECIO'
    MANAGEMENT_FEE = 0.1
    AVG_OCCUPANCY = 0.85

    def __init__(self, zone, bedrooms, acquisition_price):
        super().__init__(zone, bedrooms, acquisition_price)
        self.decoration_price = CorporateProperty.DECORATION_PRICES[self.bedrooms]
        self.equipment_price = CorporateProperty.EQUIPMENT_PRICES[self.bedrooms]
        self.utilities_price = CorporateProperty.UTILITIES_PRICES[self.bedrooms]
        self.hotelery_price = CorporateProperty.HOTELERY_PRICES[self.bedrooms]
        self.internet_price = CorporateProperty.INTERNET_PRICE
        self.management_fee = CorporateProperty.MANAGEMENT_FEE
        self.avg_occupancy = CorporateProperty.AVG_OCCUPANCY

    @property
    def average_rental_price(self):
        df = pd.read_excel(CorporateProperty.PRICE_DATASET)

        # Filter dataset by number of rooms
        df = df[df[CorporateProperty.PRICE_DATASET_ROOMS_COLUMN].isin([self.bedrooms])]

        # Filter dataset by zone
        df = df[df[CorporateProperty.PRICE_DATASET_ZONE_COLUMN].isin([self.zone])]

        # Get average price
        return df[CorporateProperty.PRICE_DATASET_PRICE_COLUMN].mean()




class TouristProperty(Property):

    RENOVATION_COST_PRICES = {1:50, 2:66, 3:83, 4:100}
    UTILITIES_PRICES = {1:120, 2:150, 3:170, 4:200}
    DECORATION_PRICES = {1:12000, 2:14000, 3:17000, 4:20000}
    EQUIPMENT_PRICES = {1:1200, 2:1400, 3:1700, 4:2000}
    HOTELERY_PRICES = {1:108, 2:150, 3:220, 4:280}
    INTERNET_PRICE = 50
    PRICE_DATASET = 'rental_prices_tourist.csv'
    PRICE_DATASET_ZONE_COLUMN = 'LOCATION'
    PRICE_DATASET_ROOMS_COLUMN = 'ROOMS'
    PRICE_DATASET_DAILY_RATE_COLUMN = 'Avg. Daily Rate'
    PRICE_DATASET_OCCUPANCY_RATE_COLUMN = 'Occupancy Rate'
    MANAGEMENT_FEE = 0.16
    AIRBNB_FEE = 0.14


    def __init__(self, zone, bedrooms, acquisition_price):
        super().__init__(zone, bedrooms, acquisition_price)
        self.decoration_price = TouristProperty.DECORATION_PRICES[self.bedrooms]
        self.equipment_price = TouristProperty.EQUIPMENT_PRICES[self.bedrooms]
        self.utilities_price = TouristProperty.UTILITIES_PRICES[self.bedrooms]
        self.hotelery_price = TouristProperty.HOTELERY_PRICES[self.bedrooms]
        self.internet_price = TouristProperty.INTERNET_PRICE
        self.management_fee = TouristProperty.MANAGEMENT_FEE
        self.airbnb_fee = TouristProperty.AIRBNB_FEE

    @property
    def average_rental_price(self):
        df = pd.read_csv(TouristProperty.PRICE_DATASET)

        # Filter dataset by number of rooms
        df = df[df[TouristProperty.PRICE_DATASET_ROOMS_COLUMN].isin([self.bedrooms])]

        # Filter dataset by zone
        df = df[df[TouristProperty.PRICE_DATASET_ZONE_COLUMN].isin([self.zone])]

        # Get average price
        return (df[TouristProperty.PRICE_DATASET_DAILY_RATE_COLUMN].mean() * 30)* df[TouristProperty.PRICE_DATASET_OCCUPANCY_RATE_COLUMN].mean()

    @property
    def avg_occupancy(self):
        df = pd.read_csv(TouristProperty.PRICE_DATASET)

        # Filter dataset by number of rooms
        df = df[df[TouristProperty.PRICE_DATASET_ROOMS_COLUMN].isin([self.bedrooms])]

        # Filter dataset by zone
        df = df[df[TouristProperty.PRICE_DATASET_ZONE_COLUMN].isin([self.zone])]

        # Get average occupancy
        return df[TouristProperty.PRICE_DATASET_OCCUPANCY_RATE_COLUMN].mean()

    @property
    def opex(self):
        opex_costs = [self.utilities_price,
                      self.internet_price, self.municipal_tax, self.renovation_cost, self.community_cost]
        return sum(opex_costs) * self.management_fee * self.airbnb_fee

    @property
    def noi(self):
        return self.average_rental_price - self.opex


class TraditionalProperty(Property):

    RENOVATION_COST_PRICES = {1:50, 2:66, 3:83, 4:100}
    UTILITIES_PRICES = {1:120, 2:150, 3:170, 4:200}
    DECORATION_PRICES = {1:12000, 2:14000, 3:17000, 4:20000}
    EQUIPMENT_PRICES = {1:1200, 2:1400, 3:1700, 4:2000}
    HOTELERY_PRICES = {1:108, 2:150, 3:220, 4:280}
    INTERNET_PRICE = 50
    PRICE_DATASET = 'rental_prices_traditional.csv'
    PRICE_DATASET_ZONE_COLUMN = 'LOCATIONNAME'
    PRICE_DATASET_ROOMS_COLUMN = 'ROOMS'
    PRICE_DATASET_PRICE_COLUMN = 'PRICE_ASKING'
    MANAGEMENT_FEE = 1
    AVG_OCCUPANCY = 1


    def __init__(self, zone, bedrooms, acquisition_price):
        super().__init__(zone, bedrooms, acquisition_price)
        self.decoration_price = TraditionalProperty.DECORATION_PRICES[self.bedrooms]
        self.equipment_price = TraditionalProperty.EQUIPMENT_PRICES[self.bedrooms]
        self.utilities_price = TraditionalProperty.UTILITIES_PRICES[self.bedrooms]
        self.hotelery_price = TraditionalProperty.HOTELERY_PRICES[self.bedrooms]
        self.internet_price = TraditionalProperty.INTERNET_PRICE
        self.management_fee = TraditionalProperty.MANAGEMENT_FEE
        self.avg_occupancy = TraditionalProperty.AVG_OCCUPANCY

    @property
    def average_rental_price(self):
        df = pd.read_csv(TraditionalProperty.PRICE_DATASET)

        # Filter dataset by number of rooms
        df = df[df[TraditionalProperty.PRICE_DATASET_ROOMS_COLUMN].isin([self.bedrooms])]

        # Filter dataset by zone
        df = df[df[TraditionalProperty.PRICE_DATASET_ZONE_COLUMN].isin([self.zone])]

        # Get average price
        return df[TraditionalProperty.PRICE_DATASET_PRICE_COLUMN].mean()

    @property
    def opex(self):
        opex_costs = [self.municipal_tax, self.renovation_cost, self.community_cost]
        return sum(opex_costs)


In [89]:
piso = CorporateProperty('Moncloa', 2, 500000)
print(piso.__dict__)
print(piso.average_rental_price)
print(piso.investment)
print(piso.noi)
print(piso.profitability)

piso = TouristProperty('Latina', 2, 500000)
print(piso.__dict__)
print(piso.average_rental_price)
print(piso.investment)
print(piso.noi)
print(piso.profitability)

piso = TraditionalProperty('Latina', 2, 500000)
print(piso.__dict__)
print(piso.average_rental_price)
print(piso.investment)
print(piso.noi)
print(piso.profitability)
## INPUTS 

# USUARIO = ZONA, PRECIO COMPRA y Nº Habitaciones

## DATASET

# Output data = alquiler medio corporativo, tradicional y turístico



## CALCULAR INGRESOS POR ALQUILERES

# tradicional = alquiler bruto medio dataset tradicional - costes (15%) = ingreso neto
# Corporativo = alquiler bruto medio dataset corporativo - costes (30%) = ingreso neto 
# turistico = alquiler bruto medio dataset turistico - costes (45%) = ingreso

## CALCULAR RENTABILIDAD NETA

# tradicional = ingreso neto tradicional / acquisition price * 100
# Corporativo = ingreso neto corporativa / acquisition price * 100
# turistico = ingreso neto turístico / acquisition price * 100


{'bedrooms': 2, 'zone': 'Moncloa', 'acquisition_price': 500000, 'municipal_tax': 79.83333333333333, 'renovation_cost': 66, 'community_cost': 150, 'internet_price': 50, 'price_dataset': None, 'management_fee': 0.1, 'decoration_price': 14000, 'equipment_price': 1400, 'utilities_price': 150, 'hotelery_price': 150, 'avg_occupancy': 0.85}
3206.896551724138
515400
2661.2787356321837
6.1962252284800545
{'bedrooms': 2, 'zone': 'Latina', 'acquisition_price': 500000, 'municipal_tax': 79.83333333333333, 'renovation_cost': 66, 'community_cost': 150, 'internet_price': 50, 'price_dataset': None, 'management_fee': 0.16, 'decoration_price': 14000, 'equipment_price': 1400, 'utilities_price': 150, 'hotelery_price': 150, 'airbnb_fee': 0.14}
1131.9576125877272
515400
1120.8509459210607
2.609664600514693
{'bedrooms': 2, 'zone': 'Latina', 'acquisition_price': 500000, 'municipal_tax': 79.83333333333333, 'renovation_cost': 66, 'community_cost': 150, 'internet_price': 50, 'price_dataset': None, 'management_fee

In [ ]:
## Investment Costs

In [ ]:
#define cost of decoration:



In [ ]:
def row_filter(df, cat_var, cat_values):
    '''
    Return: a Pandas dataframe object where columns have been filtered by a set of values from a given column (categorical variable). 
            The resulting dataframe will be sorted descending from highest to lowest amount of deaths and the index column will be reset.
    Input parameters:
        - df -> Pandas dataframe object: a dataframe with categorical variables.
        - cat_var -> string: a string with the name of a categorical variable (e.g.: 'Sexo').
        - cat_values -> list object: a list of values (string) which rows will be INCLUDED into the returned dataframe (e.g.: ['Hombres', 'Mujeres'])
    '''
    df = df[df[cat_var].isin(cat_values)].sort_values(by='Total', ascending=False)
    return df.reset_index(drop=True)


In [ ]:
def groupby_sum(df, group_vars, agg_var='Total', sort_var='Total'):
    '''
    Return: a Pandas dataframe object where rows have been gruped by a given group of columns (categorical variables). 
            The resulting dataframe will be sorted descending from highest to lowest amount of deaths and the index column will be reset.
    Input parameters:
        - df -> Pandas dataframe object: a dataframe with categorical variables and an aggregation variable.
        - group_vars -> list object: a list of values with the name of a group of categorical variables (e.g.: ['Sexo', 'Edad']).
        - agg_var -> string: a string with the name of the variable to be aggregated. In this case the variable 'Total' (number of deaths) is set as default.
        - sort_var -> string: a string with the name of the variable to sort the dataframe by. In this case the variable 'Total' (number of deaths) is set as default.
    '''
    df = df.groupby(group_vars, as_index=False).agg({agg_var:'sum'})
    df = df.sort_values(by=sort_var, ascending=False)
    return df.reset_index(drop=True)

In [ ]:
# fee of darya modern living

def fee_cost(total_income):
    return total_income * 0.1
    

In [10]:
### Business Rules for tourist rentals

In [ ]:
# Define cost of OTAS. 

def otas(total_income):
    fee_booking = 0.17
    fee_airbnb = 0.14
    occ_booking = 0.4
    occ_airbnb = 0.6
    booking = (total_income * occ_booking) * fee_booking
    airbnb = (total_income * occ_airbnb) * fee_airbnb
    return booking + airbnb



In [11]:
## making p&l:

total_income = monthly_rent * 12
total_costs = utilities() + internet() +   

NameError: name 'monthly_rent' is not defined